# ID2222 - Homework 1

You are to implement the stages of finding textually similar documents based on Jaccard similarity using the shingling, minhashing, and locality-sensitive hashing (LSH) techniques and corresponding algorithms. The implementation can be done using any big data processing framework, such as Apache Spark, Apache Flink, or no framework, e.g., in Java, Python, etc. To test and evaluate your implementation, write a program that uses your implementation to find similar documents in a corpus of 5-10 or more documents such as web pages or emails.

The stages should be implemented as a collection of classes, modules, functions or procedures depending the framework and the language of your choice. Below, we give a description of sample classes that implement different stages of finding textually similar documents. You do not have to develop the exact same classes and data types as described below. Feel free to use data structures that suit you best.

1. A class Shingling that constructs k–shingles of a given length k (e.g., 10) from a given document, computes a hash value for each unique shingle, and represents the document in the form of an ordered set of its hashed k-shingles.
2. A class CompareSets that computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.
3. A class MinHashing that builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).
4. A class CompareSignatures that estimates similarity of two integer vectors – minhash signatures – as a fraction of components, in which they agree.
5. (Optional task for extra 2 bonus) A class LSH that implements the LSH technique: given a collection of minhash signatures (integer vectors) and a similarity threshold t, the LSH class (using banding and hashing) finds all candidate pairs of signatures that agree on at least fraction t of their components.

To test and evaluate scalability (the execution time versus the size of input dataset) of your implementation, write a program that uses your classes to find similar documents in a corpus of 5-10 documents. Choose a similarity threshold s (e.g., 0,8) that states that two documents are similar if the Jaccard similarity of their shingle sets is at least s. 

## Scalability
To test and evaluate scalability (the execution time versus the size of input dataset) of your implementation, write a program that uses your classes to find similar documents in a corpus of 5-10 documents. Choose a similarity threshold s (e.g., 0,8) that states that two documents are similar if the Jaccard similarity of their shingle sets is at least s.


In [1]:
import json

def load_and_prepare_data(path):
    with open("train-v2.0.json") as f:
        d = f.read()
        data = json.loads(d)

    characters = {}
    for d in data['data']:
        info = d['paragraphs'][0]['context']

        characters[d['title']] = info

    return characters

In [2]:
import os

def load_business():
    path = "bbc-full-text-document-classification/bbc/business"
    file_list = os.listdir(path)
    
    data = {}
    
    for file in file_list:
        with open(os.path.join(path, file)) as f:
            data[file.split(".")[0]] = f.read().replace('\n', ' ').lower()
            
    return data

In [3]:
dataset = load_business()

In [4]:
#dataset = load_and_prepare_data("train-v2.0.json")

In [5]:
import binascii
import time

class Shingling:
    def __init__(self, k):
        self.k = k
        self.docs_shingles = {}
        self.doc_names = []
        
    def _clean(self, doc):
        """
        Some rules for cleaning the text:
        https://www.cs.utah.edu/~jeffp/teaching/cs5955/L4-Jaccard+Shingle.pdf
        """
        doc = doc.lower()
        doc = doc.replace(" ", "_")
        return doc
    
    def _tokenize(self, doc):
        """
        Construct the shingles based on k-characters
        """
        sh = set()
        if len(doc) >= self.k:
            for idx, token in enumerate(doc):
                if idx + self.k <= len(doc):
                    sh.add(self._hash(doc[idx:idx + self.k]))

        return sh

    def _hash(self, shingle):
        """
        Compute hash values for the shingle
        """
        return binascii.crc32(shingle.encode("utf-8")) & 0xffffffff
    
    def generate_shingles(self, doc):
        doc = self._clean(doc)
        shingles = self._tokenize(doc)
        
        return shingles  
    
    def generate_shingles_for_docs(self, docs):
        """
        Takes in docs in the form of a dict of {"docID": "doc string"}
        """
        print("Shingling {} articles...".format(len(docs)))

        t0 = time.time()
        for k, v in docs.items():
            self.doc_names.append(k)
            d = self._clean(v)
            d = self._tokenize(d)
    
            self.docs_shingles[k] = d
    
        print ('\nShingling took %.2f sec.' % (time.time() - t0))

    @staticmethod
    def compare_sets(s1, s2):
        """
        Compute Jaccard Similarity
        n(intersection) / n(union)
        """
        # add in some checks 
        jacc_sim = (len(s1.intersection(s2)) / float(len(s1.union(s2))))
        return jacc_sim

In [6]:
shing = Shingling(4)

In [7]:
shing.generate_shingles_for_docs(dataset)

Shingling 510 articles...

Shingling took 2.63 sec.


In [8]:
shing.compare_sets(shing.docs_shingles[shing.doc_names[1]],
                   shing.docs_shingles[shing.doc_names[2]])

0.14781834372217276

In [9]:
import numpy as np

def generate_jaccard_sim(docs_shingles, doc_names):
    print("Calculating the Jaccard Similarity for all documents")
    dataset_size = len(docs_shingles)

    jaccSimMatrix = np.zeros(dataset_size * dataset_size).reshape(dataset_size,dataset_size)
    t0 = time.time()

    docKeys = list(docs_shingles.keys())
    for j in range(0, len(doc_names)):    
        s1 = docs_shingles[doc_names[j]]
        for k in range(j, len(doc_names)):
            s2 = docs_shingles[doc_names[k]]
            jacc_sim = (len(s1.intersection(s2)) / float(len(s1.union(s2))))
            jaccSimMatrix[j, k] = jacc_sim
            jaccSimMatrix[k, j] = jacc_sim

    print ('\nJaccard Similarity for ' + str(len(doc_names)) + ' docs took %.2f sec.' % (time.time() - t0))

In [10]:
# jaccSimMatrix  = generate_jaccard_sim(shing.docs_shingles, shing.doc_names)

### MinHashing

In [11]:
import random 

class MinHashing:
    
    def __init__(self, n, max_shingle_ID = 2**32-1):
        self.n = n # number of hashes
        self.max_shingle_ID = max_shingle_ID # the max number
        self.next_prime = 4294967311 # the next prime number after max shingle ID
        self.coeffs_A = self.generate_coeffs()
        self.coeffs_B = self.generate_coeffs()
        self.docs_minhash_signatures = {}
    
    def generate_coeffs(self):
        """
        Create a list of 'n' unique random values.
        """
        coeffs_list = []
        
        for _ in range(self.n):
            # TODO: check if it a good idea to have 0 for coeff A
            rand_idx = random.randint(0, self.max_shingle_ID)

            # Ensure that each random number is unique.
            while rand_idx in coeffs_list:
                rand_idx = random.randint(0, self.max_shingle_ID)

            coeffs_list.append(rand_idx)

        return coeffs_list

    def _minHash_function(self, pos, x):
        """
        Return a hash in the form of (ax+b) % prime
        """
        return (self.coeffs_A[pos] * x + self.coeffs_B[pos]) % self.next_prime
        
    def generate_signature(self, shingle_set):
        """
        Given a shingle set of IDs, generate the hashes and compute the minimum hash
        """
        signature = []
        
        for i in range(self.n):
            signature.append(min(map(lambda x: self._minHash_function(i,x), shingle_set)))

        return signature
    
    def generate_doc_signatures(self, shingles):
        print("Generating MinHash signatures for documents..")
        t0 = time.time()

        for k, v in shingles.items():
            self.docs_minhash_signatures[k] = self.generate_signature(v)
       
        print ('\n Generating Signatures for ' + str(len(shingles)) + ' docs took %.2f sec.' % (time.time() - t0))

    @staticmethod
    def compare_signatures(s1, s2):
        if not len(s1) == len(s2):
            print("Unequal length of Signature")
            
        equality = 0
        signature_len = len(s1)
        for x, y in zip(s1, s2):
            if(x == y):
                equality += 1 
        return equality / float(signature_len)

In [12]:
minhash = MinHashing(100)

In [13]:
minhash.generate_signature(shing.docs_shingles[shing.doc_names[1]])

[869666,
 556469,
 3534900,
 3890139,
 5378602,
 470916,
 1322947,
 6030977,
 4909494,
 2766997,
 3888081,
 4574207,
 4569552,
 847551,
 2533533,
 2057741,
 1458543,
 148928,
 318836,
 302128,
 468841,
 7600415,
 839921,
 2079849,
 1844037,
 364503,
 737076,
 576240,
 10773298,
 389991,
 2060149,
 555564,
 852796,
 698145,
 2708435,
 10712510,
 625492,
 5101586,
 3223696,
 50099,
 866641,
 3670445,
 4118320,
 329929,
 1689569,
 355086,
 1310794,
 3570832,
 290250,
 4078725,
 1352298,
 206529,
 5302739,
 1690114,
 1254015,
 1321394,
 3877513,
 3406763,
 2888095,
 2532844,
 4689460,
 1625310,
 5338377,
 3575923,
 2591847,
 2270034,
 8891202,
 420527,
 1753523,
 1150765,
 207593,
 3502670,
 218353,
 1729106,
 71995,
 870603,
 3230584,
 5711224,
 5675339,
 10251265,
 85060,
 548180,
 2122877,
 2497270,
 7754836,
 2450101,
 2081333,
 3063637,
 4199246,
 645335,
 614416,
 8294555,
 779118,
 2612049,
 202906,
 677139,
 1530834,
 336051,
 3688991,
 1236558]

In [14]:
minhash.compare_signatures(minhash.generate_signature(shing.docs_shingles[shing.doc_names[1]]),
                   minhash.generate_signature(shing.docs_shingles[shing.doc_names[2]]))

0.11

In [15]:
minhash.generate_doc_signatures(shing.docs_shingles)

Generating MinHash signatures for documents..

 Generating Signatures for 510 docs took 63.51 sec.


### LSH
Partition into Bands
- Divide matrix M into b bands of r rows.
- For each band, hash its portion of each column to a hash table with k buckets.
- Make k as large as possible.
- Candidate column pairs are those that hash to the same bucket for a number of bands with regards to the threshold set.

In [16]:
from collections import defaultdict

class LSH:
    
    def __init__(self, band_size, row_size, threshold):
        self.band_size = band_size
        self.threshold = threshold
        self.row_size = row_size
        self.docs_lsh = {}
        self.candidate_pairs = defaultdict(set)

    def get_lsh(self, signature):
        lsh = []
        for i in range(self.band_size):
            lsh.append(hash(tuple(signature[i*self.row_size:(i*self.row_size+self.row_size)])) % 4294967311)
        return lsh

    def get_lsh_for_docs(self, signatures):
        for k, v in signatures.items():
            self.docs_lsh[k] = self.get_lsh(v)
        
    def generate_candidate_pairs(self, t=0.0002):
        """
        t: the fraction of components that pair of signatures agrees on
        """
        all_docs = list(self.docs_lsh.values())
        all_names = list(self.docs_lsh.keys())
        
        # Minimum number of bands that should has overlap
        # hash according to the threshold set
        threshold = t * self.band_size

        # Stores the intermediate number of band overlaps
        pairs = defaultdict(lambda: defaultdict(float))

        for idx, s1 in enumerate(all_docs):
            s1_name = all_names[idx]
            
            # Sliding count to perform comparison
            for curr_iter, s2 in enumerate(all_docs[idx + 1:]):
                s2_name = all_names[curr_iter + idx + 1]
                
                for x, y in zip(s1, s2):
                    if(x == y):
                        if not pairs[s1_name][s2_name]:
                            pairs[s1_name][s2_name] = 1
                        else:
                            pairs[s1_name][s2_name] += 1

                # Store pairs that is above the threshold as candidate pairs
                if pairs[s1_name][s2_name] > threshold:
                    self.candidate_pairs[s1_name].add(s2_name)

In [17]:
lshh = LSH(10, 10, 0.8)

In [18]:
lshh.get_lsh(minhash.docs_minhash_signatures[shing.doc_names[1]])

[3095441960,
 823489805,
 4257584683,
 3143060384,
 974132344,
 814869834,
 2755579486,
 801032633,
 3117601178,
 3867275264]

In [19]:
lshh.get_lsh_for_docs(minhash.docs_minhash_signatures)

In [20]:
lshh.docs_lsh[shing.doc_names[1]]

[3095441960,
 823489805,
 4257584683,
 3143060384,
 974132344,
 814869834,
 2755579486,
 801032633,
 3117601178,
 3867275264]

In [21]:
lshh.generate_candidate_pairs()

In [22]:
lshh.candidate_pairs

defaultdict(set,
            {'007': {'253'},
             '214': {'240'},
             '215': {'494'},
             '229': {'209'},
             '258': {'356'},
             '260': {'354'},
             '265': {'333'},
             '267': {'365'},
             '371': {'256'},
             '416': {'292'},
             '438': {'290'},
             '493': {'286'}})

In [25]:
print("This article:\n\n", dataset['229'])
print("\n\nis similar to:\n ", dataset['209'])

This article:

 shares rise on new man utd offer  shares in manchester united closed up 4.75% on monday following a new offer from us tycoon malcolm glazer.  the board of the football club is expected to meet early this week to discuss the latest proposal, which values the club at £800m ($1.5bn). manchester united revealed on sunday that it had received a detailed proposal from mr glazer, which looks set to receive more serious scrutiny. the club has previously rejected mr glazer's approaches out of hand. but a senior source at the club told the bbc: "this time it's different." supporters' group shareholders united, however, urged the club to reject the new deal.  a spokesman for the shareholders united said: "i can't see any difference (compared to mr glazer's previous proposals) other than £200m less debt. "he isn't bringing any money into the club; he'll use our money to buy it."  mr glazer's latest move is being led by mr glazer's two sons, avi and joel, according to the financial 